In [1]:
# bibliotecas utilizadas:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import sys
import os
from pathlib import Path
import re
from pprint import pprint
from tqdm import tqdm
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.colors as mcolors
%matplotlib inline

In [2]:
# Gensim (LDA)
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

C:\Users\gusta\anaconda3\Lib\site-packages\botocore\utils.py:15: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi
C:\Users\gusta\anaconda3\Lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [3]:
# lista para armazenar caminho de pastas
folder_paths = ["C:/Users/gusta/Documents/OneDrive - SABESP/Data Models/Imprensa/Modelo_RiscoImprensa/Base"]
# Lista para armazenar arquivos excel
excel_files = []
# Laço para capturar os arquivos do tipo xmls
for folder_path in folder_paths:
    for file in os.listdir(folder_path):
        if file.endswith(".xlsx"):  # Certifique-se de ajustar a extensão do arquivo se necessário
            excel_files.append(os.path.join(folder_path, file))

In [4]:
# lista para armazenar data frames
# laço para criar dataframe para cada arquivo armazenado
dataframes = []
for excel_file in excel_files:
    df = pd.read_excel(excel_file)
    dataframes.append(df)

# concatenar dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\gusta\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarnin

In [5]:
# análise prévia do material
combined_df.head()

,Data,Título,Link original,Veículo,Mídia geral,Tema,Sub-tema,Mensagem,Projeto Citado,Tom Auditoria,Sentimento Cena
0,2023-01-01 00:00:00,"""Primeiro vou cuidar das pessoas"", diz novo go...",https://bookcontent.boxnet.com.br/Visualizar/?...,DIÁRIO DE SUZANO,Web,Privatização ou capitalização,Privatização ou Capitalização,Não,NaN,Neutro,Favorável
1,2023-01-01 00:00:00,"“Primeiro vou cuidar das pessoas”, diz novo go...",https://bookcontent.boxnet.com.br/Visualizar/?...,ARARAQUARA NEWS / ARARAQUARA,Web,Privatização ou capitalização,Privatização ou Capitalização,Não,NaN,Neutro,Favorável
2,2023-01-01 00:00:00,"“Primeiro vou cuidar das pessoas”, diz novo go...",https://bookcontent.boxnet.com.br/Visualizar/?...,DIÁRIO DA REGIÃO ONLINE / SÃO JOSÉ DO RIO PRETO,Web,Privatização ou capitalização,Privatização ou Capitalização,Não,NaN,Neutro,Favorável
3,2023-01-01 00:00:00,"“Primeiro vou cuidar das pessoas”, diz novo go...",https://bookcontent.boxnet.com.br/Visualizar/?...,ISTOÉ ONLINE / SÃO PAULO,Web,Privatização ou capitalização,Privatização ou Capitalização,Não,NaN,Neutro,Favorável
4,2023-01-01 00:00:00,Chuva forte causa vazamento de esgoto em Suzan...,https://bookcontent.boxnet.com.br/Visualizar/?...,G1 / NACIONAL,Web,Esgoto,Vazamentos,Não,NaN,Negativo,Negativo


In [6]:
combined_df['Tema'].value_counts()

Tema
Privatização ou capitalização       4021
Abastecimento de água               3058
Reparos / vazamentos / obras        2473
Institucional                       2411
Financeiro e novos negócios         1151
Saneamento                          1120
Eventos, patrocínios e parcerias     820
Esgoto                               581
Mananciais                           548
Novas tecnologias e inovação         187
Uso racional da água                  42
Name: count, dtype: int64

In [7]:
combined_df['Sub-tema'].value_counts()

Sub-tema
Privatização ou Capitalização               3155
Falta de água/Litoral Norte                  774
Privatização ou Capitalização / Eleições     672
Bolsa de Valores                             651
Desabastecimento                             432
                                            ... 
Bica                                           1
Conta - Acordo para pagamento                  1
Obras / Enchentes                              1
ESG - mulheres em cargos de lideranca          1
CEI Sabesp                                     1
Name: count, Length: 991, dtype: int64

In [8]:
# análise de nulls
combined_df.isnull().sum()

Data                   0
Título                 0
Link original          0
Veículo                0
Mídia geral            0
Tema                   0
Sub-tema               0
Mensagem               0
Projeto Citado     15457
Tom Auditoria          0
Sentimento Cena        0
dtype: int64

In [9]:
# análise de dados descritivos
combined_df.describe()

,Data,Título,Link original,Veículo,Mídia geral,Tema,Sub-tema,Mensagem,Projeto Citado,Tom Auditoria,Sentimento Cena
count,16412,16412,16412,16412,16412,16412,16412,16412,955,16412,16412
unique,367,12339,16403,1299,4,11,991,3,85,4,4
top,2023-04-10 00:00:00,Destaques,https://www.assiscity.com/local/ultimos-dias-d...,"RÁDIO BANDNEWS FM 96,9 / SÃO PAULO",Web,Privatização ou capitalização,Privatização ou Capitalização,Não,Novo Rio Pinheiros,Neutro,Favorável
freq,316,109,2,413,6856,4021,3155,15339,156,8229,8399


In [10]:
# verificando o conteúdo da coluna Projeto Citado
print(combined_df["Projeto Citado"].value_counts().head(10))

Projeto Citado
Novo Rio Pinheiros                          156
Campanha do Agasalho 2023                   143
Programa Aprendiz 2023                      123
Acertando Suas Contas com a Sabesp           68
Óleo do Bem                                  65
Onda Limpa                                   40
Campanha Lacres e Tampinhas de Esperança     36
Projeto Tietê                                34
Integra Tietê                                21
Programa Detecta                             18
Name: count, dtype: int64


In [11]:
# excluindo a coluna projeto citado
combined_df = combined_df.drop("Projeto Citado", axis=1)
combined_df.columns

Index(['Data', 'Título', 'Link original', 'Veículo', 'Mídia geral', 'Tema',
       'Sub-tema', 'Mensagem', 'Tom Auditoria', 'Sentimento Cena'],
      dtype='object')

In [12]:
combined_df.dtypes

Data               object
Título             object
Link original      object
Veículo            object
Mídia geral        object
Tema               object
Sub-tema           object
Mensagem           object
Tom Auditoria      object
Sentimento Cena    object
dtype: object

In [13]:
combined_df['Data'] = pd.to_datetime(combined_df['Data'])
combined_df.Data

0       2023-01-01
1       2023-01-01
2       2023-01-01
3       2023-01-01
4       2023-01-01
           ...    
16407   2022-12-31
16408   2022-12-31
16409   2022-12-31
16410   2022-12-31
16411   2022-12-31
Name: Data, Length: 16412, dtype: datetime64[ns]

In [14]:
combined_df.Data.max(), combined_df.Data.min()

(Timestamp('2023-07-31 00:00:00'), Timestamp('2022-08-01 00:00:00'))

In [15]:
# Definindo a ordem pela data
combined_df = combined_df.sort_values(by='Data')
combined_df.Data.head(3), combined_df.Data.tail(3)

(11451   2022-08-01
 11452   2022-08-01
 11440   2022-08-01
 Name: Data, dtype: datetime64[ns],
 11327   2023-07-31
 11319   2023-07-31
 11438   2023-07-31
 Name: Data, dtype: datetime64[ns])

In [16]:
# extrair uma tupla apenas com os títulos
df_titulo = combined_df['Título']
pd.df_titulo = df_titulo
df_titulo.head()

11451          Represas da região: Nível nos últimos meses
11452    Volume de chuva em julho foi o menor do ano na...
11440    Moradores da Vila Aurora e Santa Luzia sofrem ...
11463                      Mensagens dos telespectadores 2
11462    Bueiro vaza esgoto e incomoda moradores do Jar...
Name: Título, dtype: object

In [17]:
# qtde de linhas repetidos
rep_rows = df_titulo.shape[0] - df_titulo.nunique()
print(df_titulo.shape, '\n', 
      df_titulo.nunique(), '\n',
      rep_rows)

(16412,) 
 12339 
 4073


## FAZER UMA ANÁLISE ESPECIFICA DOS TITULOS REPETIDA

In [18]:
# dropar linhas repetidas e checar
df_titulos = df_titulo.drop_duplicates()
print(df_titulos.shape, df_titulos.nunique())

(12339,) 12339


In [19]:
df_titulos = pd.DataFrame(df_titulos)

In [20]:
df_titulos.head()

,Título
11451,Represas da região: Nível nos últimos meses
11452,Volume de chuva em julho foi o menor do ano na...
11440,Moradores da Vila Aurora e Santa Luzia sofrem ...
11463,Mensagens dos telespectadores 2
11462,Bueiro vaza esgoto e incomoda moradores do Jar...


In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gusta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
# Pré-processamento dos títulos
# Isso inclui tokenização, remoção de stopwords e outras etapas de limpeza
stop_words = set(stopwords.words('portuguese'))  # Lista de stopwords em português
texts = []  # Lista para armazenar os títulos pré-processados

In [23]:
# laço para o pré processamento
for title in df_titulos['Título']:
    tokens = word_tokenize(title.lower())  # Tokenize e coloque em minúsculas
    tokens = [word for word in tokens if word.isalpha()]  # Remova pontuações
    tokens = [word for word in tokens if word not in stop_words]  # Remova stopwords
    texts.append(tokens)


In [24]:
texts

[['represas', 'região', 'nível', 'últimos', 'meses'],
 ['volume',
  'chuva',
  'julho',
  'menor',
  'ano',
  'represas',
  'alto',
  'tietê',
  'índice',
  'representa',
  'esperado'],
 ['moradores',
  'vila',
  'aurora',
  'santa',
  'luzia',
  'sofrem',
  'racionamento',
  'água',
  'durante',
  'noite'],
 ['mensagens', 'telespectadores'],
 ['bueiro',
  'vaza',
  'esgoto',
  'incomoda',
  'moradores',
  'jardim',
  'bonsucesso',
  'novamente'],
 ['sabesp', 'ação', 'fecha', 'alta', 'nesta', 'segunda', 'veja', 'valores'],
 ['prefeitura',
  'sabesp',
  'anunciam',
  'ligação',
  'sistema',
  'lourenço',
  'roque'],
 ['autuados', 'receptação', 'falta', 'licença', 'bertioga', 'sp'],
 ['estudante',
  'prudentino',
  'arrecada',
  'mil',
  'tampinhas',
  'plástico',
  'doar',
  'he'],
 ['estudante',
  'pp',
  'arrecada',
  'mil',
  'tampinhas',
  'plástico',
  'doar',
  'hospital',
  'esperança'],
 ['btg', 'pactual', 'inclui', 'eletrobras', 'vibra', 'carteira', 'agosto'],
 ['após',
  'elet

In [25]:
# Crie um dicionário a partir dos títulos pré-processados
# dictionary = corpora.Dictionary(texts)
# print(dictionary)

Dictionary<9690 unique tokens: ['meses', 'nível', 'região', 'represas', 'últimos']...>


In [26]:
# Crie um corpus a partir do dicionário
corpus = [dictionary.doc2bow(text) for text in texts]

In [27]:
# Crie o modelo LDA
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [28]:
# Imprima os tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico #{idx}: {topic}\n')

Tópico #0: 0.067*"sabesp" + 0.023*"mutirão" + 0.019*"dívidas" + 0.018*"água" + 0.015*"inicia" + 0.014*"negociação" + 0.012*"hoje" + 0.012*"realiza" + 0.010*"poupatempo" + 0.009*"dia"

Tópico #1: 0.027*"sabesp" + 0.027*"r" + 0.027*"saneamento" + 0.023*"entrevista" + 0.012*"anuncia" + 0.012*"petrobras" + 0.010*"empresas" + 0.009*"parte" + 0.009*"milhões" + 0.009*"ação"

Tópico #2: 0.047*"sabesp" + 0.026*"sobre" + 0.016*"água" + 0.014*"conta" + 0.011*"franca" + 0.010*"ouvinte" + 0.009*"campanha" + 0.008*"partir" + 0.008*"nota" + 0.007*"procon"

Tópico #3: 0.030*"água" + 0.023*"litoral" + 0.020*"moradores" + 0.020*"sp" + 0.019*"norte" + 0.019*"sabesp" + 0.013*"esgoto" + 0.012*"obras" + 0.012*"rua" + 0.011*"falta"

Tópico #4: 0.050*"tarcísio" + 0.044*"governo" + 0.041*"sp" + 0.036*"sabesp" + 0.035*"privatização" + 0.023*"paulo" + 0.022*"freitas" + 0.019*"governador" + 0.017*"diz" + 0.011*"estado"



Rodar com 11 temas (igual número ao outro)


CÓDIGO DA CARLA

In [29]:
#atribui lista pela coluna de análise desejada

list_noticias = df_titulos['Título'].values.tolist()
#list_noticias = df_noticias['Texto Notícia limpo'].values.tolist()

In [30]:
list_noticias

['Represas da região: Nível nos últimos meses',
 'Volume de chuva em julho foi o menor do ano nas represas do Alto Tietê; índice representa 23,2% do esperado',
 'Moradores da Vila Aurora e Santa Luzia sofrem com racionamento de água durante a noite',
 'Mensagens dos telespectadores 2',
 'Bueiro vaza esgoto e incomoda moradores do Jardim Bonsucesso novamente',
 'Sabesp: ação fecha em alta de 0,34% nesta segunda; veja valores',
 'Prefeitura e Sabesp anunciam ligação do Sistema São Lourenço para São Roque',
 'Ferros-velhos são autuados por receptação e falta de licença em Bertioga, SP',
 'Estudante prudentino arrecada 13 mil tampinhas de plástico para doar ao HE',
 'Estudante de PP arrecada 13 mil tampinhas de plástico para doar ao Hospital de Esperança',
 'BTG PACTUAL INCLUI ELETROBRAS E VIBRA EM CARTEIRA PARA AGOSTO',
 'Após Eletrobras, mercado aposta na privatização da Copasa; veja cenário para estatais de saneamento',
 'Bairros de Guarujá não têm água',
 'Movimento contra a Taxa do Li

In [31]:
# lista para retirar palavras comuns e óbvias
lista_palavras_comuns = ['governo',
                         'sabesp',
                         'empresa',
                         'sao paulo',
                         'sp',
                         'companhia',
                         'ser',
                         'estado',
                         'ainda',
                         'mês',
                         'sobre',
                         'desde',
                         'disse',
                         'segundo',
                         'pode',
                         'ano',
                         'mil',
                         'milhões',
                         'dia',
                         'nesta',
                         'hoje',
                         'paulo',
                         'janeiro',
                         'fevereiro',
                         'março',
                         'abril',
                         'maio',
                         'junho',
                         'julho',
                         'agosto',
                         'setembro',
                         'outubro',
                         'novembro',
                         'dezembro',
                         'segunda',
                         'terça',
                         'quarta',
                         'quinta',
                         'sexta',
                         'sábado',
                         'domingo',
                         'feira',
                         'além',
                         'disso',
                         'empresas',
                         'desse',
                         'conteúdo',
                         'redação'
]

In [32]:
#resolvendo alguns problemas de codificação
for index, x in enumerate(list_noticias):
    x = str(x)
    if '\x84' in x:
      list_noticias[index] = x.replace('\x84', 'ä')
    if '\x80' in x:
      list_noticias[index] = x.replace('\x80', 'ç')